## Prerequisites
1. Creating a three ENV variables in the jupyter service :
    * GIT_TOKEN
    * AWS_ACCESS_KEY_ID
    * AWS_SECRET_ACCESS_KEY
2. clone this repo to your jupyter service - make sure you are running this notebook from the repo directory 


In [ ]:
import mlrun

In [ ]:
from mlrun.artifacts import DatasetArtifact,Artifact

In [ ]:
project = mlrun.load_project('./')

In [ ]:
project.save()

#### Project YAML

In [ ]:
dataset = project.get_artifact('cancer-dataset')

In [ ]:
job_function_run = project.run_function('fetch-data',inputs={'dataset' : dataset.target_path},local=True)

In [ ]:
job_function = project.get_function('fetch-data',sync=True).apply(mlrun.platforms.mount_s3())

In [ ]:
job_function_run = job_function.run(inputs={'dataset' : dataset.target_path})

In [ ]:
job_function_run.outputs

In [ ]:
trainer = project.get_function('trainer').apply(mlrun.platforms.mount_s3())

In [ ]:
trainer_run = trainer.run(inputs = {"dataset":job_function_run.outputs['dataset']},params = {"n_estimators": 100, "learning_rate": 1e-1, "max_depth": 3})

In [ ]:
serving = project.get_function('serving').apply(mlrun.platforms.mount_s3())

In [ ]:
serving_func = project.deploy_function('serving',models=[{'key':'cancer-classifier','model_path':trainer_run.outputs["model"], 'class_name':'mlrun.frameworks.sklearn.SklearnModelServer'}])

In [ ]:
my_data = {"inputs"
           :[[
               1.371e+01, 2.083e+01, 9.020e+01, 5.779e+02, 1.189e-01, 1.645e-01,
               9.366e-02, 5.985e-02, 2.196e-01, 7.451e-02, 5.835e-01, 1.377e+00,
               3.856e+00, 5.096e+01, 8.805e-03, 3.029e-02, 2.488e-02, 1.448e-02,
               1.486e-02, 5.412e-03, 1.706e+01, 2.814e+01, 1.106e+02, 8.970e+02,
               1.654e-01, 3.682e-01, 2.678e-01, 1.556e-01, 3.196e-01, 1.151e-01]
            ]
}
serving_func.function.invoke("/", body=my_data)

In [ ]:
project.run(workflow_path='workflow.py',watch=True)